In [1]:
from ray_trading_env import RayTradingEnvironment

INFO:ray_trading_env:ray_trading_env logger started.


In [2]:
import numpy as np
import pandas as pd
import pprint
import ray
from ray.rllib.agents.ppo import PPOTrainer

In [3]:
# Start a new instance of Ray
# Note: execute cell only once!
# instance can be stopped with ray.shutdown()
ray.init()

2022-06-27 18:05:18,618	WARNING services.py:2002 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67096576 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.


RayContext(dashboard_url='', python_version='3.8.8', ray_version='1.13.0', ray_commit='e4ce38d001dbbe09cd21c497fedd03d692b2be3e', address_info={'node_ip_address': '10.1.150.226', 'raylet_ip_address': '10.1.150.226', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-06-27_18-05-16_985733_54509/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-06-27_18-05-16_985733_54509/sockets/raylet', 'webui_url': '', 'session_dir': '/tmp/ray/session_2022-06-27_18-05-16_985733_54509', 'metrics_export_port': 54249, 'gcs_address': '10.1.150.226:59171', 'address': '10.1.150.226:59171', 'node_id': '4b351714c58060216ca40ed5324db100c4ec659280f3e2b94e96430e'})

In [8]:
# config dict for Rllib
config = {
    "env": RayTradingEnvironment,
    "env_config": {
        "config": {
            "trading_days": 252,
            "trading_cost_bps": 1e-3,
            "time_cost_bps": 1e-4,
            "ticker": "AAPL",
            "get_data_from_csv": True,
            # note: need to specify path when using Tune
            "data_path": "/home/jovyan/machine-learning-for-trading/AAPL_prices.csv",
            #"max_episode_steps": 252,
        },
    },

    "create_env_on_driver": True,
    # horizon needs to be specified if the env has no 
    # max_number_of_steps-like parameter
    "horizon" : 252,
}

In [9]:
# Instantiate the Trainer object using above config.
rllib_trainer = PPOTrainer(config=config)

INFO:ray_trading_env:got data for AAPL...
/opt/conda/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
2022-06-27 18:06:35,475	WARNING env.py:135 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
2022-06-27 18:06:35,947	WARNING util.py:65 -- Install gputil for GPU system monitoring.
(RolloutWorker pid=55601) INFO:ray_trading_env:ray_trading_env logger started.
(RolloutWorker pid=55601) INFO:ray_trading_env:got data for AAPL...
(RolloutWorker pid=55601) /opt/conda/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
(RolloutWorker pid=55601)   logger.warn(
(RolloutWorker pid=55601) 2022-06-27 18:06:37,790	WARNING env.py:135 -- Your env doesn't h

In [10]:
# check the configuration of the trainer instance
rllib_trainer.get_config()

{'num_workers': 2,
 'num_envs_per_worker': 1,
 'create_env_on_driver': True,
 'rollout_fragment_length': 200,
 'batch_mode': 'truncate_episodes',
 'gamma': 0.99,
 'lr': 5e-05,
 'train_batch_size': 4000,
 'model': {'_use_default_native_models': False,
  '_disable_preprocessor_api': False,
  '_disable_action_flattening': False,
  'fcnet_hiddens': [256, 256],
  'fcnet_activation': 'tanh',
  'conv_filters': None,
  'conv_activation': 'relu',
  'post_fcnet_hiddens': [],
  'post_fcnet_activation': 'relu',
  'free_log_std': False,
  'no_final_linear': False,
  'vf_share_layers': False,
  'use_lstm': False,
  'max_seq_len': 20,
  'lstm_cell_size': 256,
  'lstm_use_prev_action': False,
  'lstm_use_prev_reward': False,
  '_time_major': False,
  'use_attention': False,
  'attention_num_transformer_units': 1,
  'attention_dim': 64,
  'attention_num_heads': 1,
  'attention_head_dim': 32,
  'attention_memory_inference': 50,
  'attention_memory_training': 50,
  'attention_position_wise_mlp_dim': 32,


In [7]:
# train the trainer instance
for _ in range(10):
    results = rllib_trainer.train()
    print(f"Iteration={rllib_trainer.iteration}: R(\"return\")={results['episode_reward_mean']}")

2022-06-27 18:05:59,117	WARNING deprecation.py:46 -- DeprecationWarning: `slice` has been deprecated. Use `SampleBatch[start:stop]` instead. This will raise an error in the future!


Iteration=1: R("return")=-0.18405314968542666
Iteration=2: R("return")=-0.15820936434258911
Iteration=3: R("return")=-0.17436414736386713
Iteration=4: R("return")=-0.1419247428548519
Iteration=5: R("return")=-0.1415584008013817
Iteration=6: R("return")=-0.15469522430710514
Iteration=7: R("return")=-0.20174449143112538
Iteration=8: R("return")=-0.18078945673885868


KeyboardInterrupt: 